# Naive Bayes Classifier

Welcome to your next lab! You will build Naive Bayes Classifier.

You will classify spam/ham messages.

**You will learn to:**
- Build the general architecture of a learning algorithm with OOP in mind:
    - Helper functions
        - Preprocessing data
        - Calculate prior probs for classes
        - Calculate likelihood probs of each word in a class
    - Main Model Class
        - Training
        - Prediction


> **Important note:** Before submission make sure that you **didn't add or delete any notebook cells**. Otherwise your work may not be accepted by the validator!

## 0 - Download data

In [1]:
!pip install wget
import wget

wget.download(
    'https://dru.fra1.digitaloceanspaces.com/DS_Fundamentals/datasets/04_supervised_learning/Naive_Bayes_Classifier/spam.csv')

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=52157771c7fbf3cd2f8cf61a661e4f9e05283b351903682edc6f01c30e748a59
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


'spam.csv'

## 1 - Packages ##

First, let's run the cell below to import all the packages that you will need during this assignment.
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [pandas](https://pandas.pydata.org/) is a library providing a convenient work with data.
- [re](https://docs.python.org/3/library/re.html) is for regex

In [2]:
import pandas as pd
import numpy as np
import re

## 2 - Overview of the Problem set ##

**Problem Statement**: You are given a dataset  containing:

* a training set of `m_train` examples
* a test set of `m_test` examples
* each example is a message that belongs to a particular class: ham or spam.

Let's get more familiar with the dataset. Load the data by running the following code.

We won't divide our data to features(X) and target(Y) here, because we need to preprocess it in a special way.

In [3]:
# Loading the data

def load_data():
    df = pd.read_csv('spam.csv', encoding='latin-1')
    df_for_tests = df.head()

    idx = np.arange(df.shape[0])
    np.random.shuffle(idx)

    train_set_size = int(df.shape[0] * 0.8)

    train_set = df.loc[idx[:train_set_size]]
    test_set = df.loc[idx[train_set_size:]]

    return train_set, test_set, df_for_tests

In [4]:
train_set, test_set, df_for_tests = load_data()

## 3 - Naive Bayes Classifier
**Mathematical expression of the algorithm**:


This algorithm is based on Bayes' theorem:
    \begin{equation}
    P(A_{j}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{1},\dots,x_{n}) = \frac{P(x_{1},\dots,x_{n}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})P(A_{j})}{P(x_{1},\dots,x_{n})}
    \end{equation}
    
Ignoring denominator (because it stays the same for all cases):

$$ \begin{equation}
    P(A_{j})P(x_{1},\dots,x_{n}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j}) = P(A_{j}, x_{1},\dots,x_{n}) = \\
  \hspace{1cm} = P(x_{1}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{2},\dots,x_{n}, A_{j})P(x_{2}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{3}, \dots ,x_{n}, A_{j})\dots P(x_{n-1}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{n}, A_{j}) \approx \\
  \hspace{1cm}
  \end{equation}$$
By making an assumption that the $x_{i}$ are conditionally independent of each other:
$$ \begin{equation} \approx P(A_{j}) \prod_{i=1}^{n} P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})
   \end{equation}$$
   
We can calculate the probability, if we know the prior probability:

$$ \begin{equation}
    y^{*} = \operatorname*{arg\,max}_{j} \big(P(A_{j}) \prod_{i=1}^{n} P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})\big)
   \end{equation}$$
   
   
Due to floating point underflow, the above is usually replaced with the numerically tractable expression:

$$ \begin{equation}
    y^{*} = \operatorname*{arg\,max}_{j} \big( \ln(P(A_{j})) + \sum_{i=1}^{n} \ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})) \big)
   \end{equation}$$
   
For more consistent knowledge of the NBC algorithm, we highly recommend you to read [this Stanford University article](https://web.stanford.edu/~jurafsky/slp3/4.pdf)

**Training the Naive Bayes Classifier**:

How can we find the probabilities $\ln(P(A_{j}))$ and $\ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j}))$ ? We'll simply use the frequencies in the data. For the class prior $P(A_{j})$ we find what percentage of the messages in our training set are in each class $A_{j}$:

$$ \begin{equation}
    \ln(P(A_{j})) = \ln\big(\frac{N_{A_{j}}}{N}\big)
    \tag{1}
   \end{equation}$$

where $N_{A_{j}}$ is the number of messages in our training data with class $A_{j}$ and $N$ be the total number of messages.

In $P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})$ we just compute as the fraction of times the word $x_{i}$ appears among all words in all messages of class $A_{j}$:

$$ \begin{equation}
    \ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})) = \ln\big(\frac{ N_{x_i, A_j}}{|A_{j}|} \big)
    \tag{2}
   \end{equation}
$$

where $N_{x_i, A_j}$ is number of times word $x_{i}$ appears in messages from class $A_{j}$ and $|A_{j}|$ - total count of all words in class $A_{j}$.

   
#### Laplace smoothing

In statistics, additive smoothing, also called Laplace smoothing, or Lidstone smoothing, is a technique that is used to smooth categorical data. Given an observation
$\begin{equation}
    message = (x_{1}\, \dots \,x_{n})
 \end{equation}$, a "smoothed" version of the data gives the estimator:


$$ \begin{equation}
    \ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})) = \ln\big(\frac{ N_{x_i, A_j} + \alpha}{ |A_{j}| +  \alpha * |V|} \big)
    \tag{3}
   \end{equation}
$$

where the pseudocount
$\begin{equation}
    \alpha > 0
 \end{equation}$ is the smoothing parameter (
$\begin{equation}
    \alpha = 0
 \end{equation}$ corresponds to no smoothing) and $V$ is a vocabulary, which consists of the union of all the words in all classes.

### 3.1 - Preprocessing the data

Our data consists of different messages. Messages contain some excess symbols, which don't affect the content of the text, but add noise to the data.
For example: "Does not \\operate 66.7 after  & lt;# & gt;  or what".

Let's clean our data and leave only letters a-z, A-Z, numbers 0-9 and cast all letters to lowercase, replace double to n spaces with just one space, remove trailing spaces.

In [5]:
# Clean the data

def clean_data(message):
    """
    Returns string which consists of message words

    Argument:
    message -- message from dataset;
        type(message) -> <class 'str'>

    Returns:
    result -- cleaned message, which contains only letters a-z, and numbers 0-9, with only one space between words;
        type(clean_data(message)) -> <class 'str'>

    """

    ### START CODE HERE ###
    message.replace('/\s+/g', ' ')
    cleaned = " ".join(re.findall(r'\w+', message))
    return cleaned.lower()
    ### END CODE HERE ###

In [6]:
sentence_1 = 'Doesn\'t get, how{to}% \\operate+66.7 :after[it]"" & lt;# & gt; won\'t `or(what)'
sentence_2 = 'O\]k,.lar7i$double{} check wif*& da! hair: [dresser;   ..already He SaID-77.88.5 wun cut v short question(std txt rate)T&C\'s'
print('cleaned: ', clean_data(sentence_1))
print('cleaned: ', clean_data(sentence_2))

cleaned:  doesn t get how to operate 66 7 after it lt gt won t or what
cleaned:  o k lar7i double check wif da hair dresser already he said 77 88 5 wun cut v short question std txt rate t c s


**Expected Output**:

<table style="width:70%">
    <tr>
        <td><b>cleaned:</b></td>
       <td> doesn t get how to operate 66 7 after it lt gt won t or what </td>
    </tr>
    <tr>
        <td><b>cleaned:</b></td>
       <td> o k lar7i double check wif da hair dresser already he said 77 88 5 wun cut v short question std txt rate t c s </td>
    </tr>
    

</table>

Now let's clean each sentence and split data on features(X) and target(Y)

In [7]:
# Preparation data for model

def prep_for_model(train_set, test_set):
    """
    Returns arrays of train/test features(words) and train/test targets(labels)

    Arguments:
    train_set -- train dataset, which consists of train messages and labels;
        type(train_set) -> pandas.core.frame.DataFrame
    test_set -- test dataset, which consists of test messages and labels;
        type(train_set) -> pandas.core.frame.DataFrame

    Returns:
    train_set_x -- array which contains lists of words of each cleaned train message;
        (type(train_set_x) ->numpy.ndarray[list[str]], train_set_x.shape = (num_messages,))
    train_set_y -- array of train labels (names of classes),
        (type(train_set_y) -> numpy.ndarray, train_set_y.shape = (num_messages,))
    test_set_x -- array which contains lists of words of each cleaned test message;
        (type(test_set_x) numpy.ndarray[list[str]], test_set_x.shape = (num_messages,)
    test_set_y -- array of test labels (names of classes),
        (type(test_set_y) -> numpy.ndarray, test_set_y.shape = (num_messages,))

    """

    ### START CODE HERE ###
    train_set_x = [clean_data(a).split() for a in (train_set['v2'].tolist())]
    train_set_y = [clean_data(a) for a in (list(train_set['v1']))]
    test_set_x = [clean_data(a).split() for a in (list(test_set['v2']))]
    test_set_y = [clean_data(a) for a in (list(test_set['v1']))]
    train_set_x = np.asarray(train_set_x, dtype='object')
    train_set_y = np.asarray(train_set_y, dtype='object')
    test_set_x = np.asarray(test_set_x, dtype='object')
    test_set_y = np.asarray(test_set_y, dtype='object')
    ### END CODE HERE ###

    return train_set_x, train_set_y, test_set_x, test_set_y


train_set_x, train_set_y, test_set_x, test_set_y = prep_for_model(train_set, test_set)

In [8]:
a1, a2, b1, b2 = prep_for_model(df_for_tests.head(3), df_for_tests.tail(2))
print(a2[0], a1[0])
print(b2[0], b1[0])

ham ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
ham ['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say']


**Expected Output**:

<table style="width:40%">
    <tr>
        <td><b>ham:</b></td>
       <td> ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat'] </td>
    </tr>
    <tr>
        <td><b>ham:</b></td>
       <td> ['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say']
 </td>
    </tr>

</table>

Now let's check words in each category

In [9]:
# Check words in categories

def categories_words(x_train, y_train):
    """
    Returns arrays of features(words) in each category and in both categories

    Arguments:
    x_train -- array which contains lists of words of each cleaned train message;
        (type(x_train) -> numpy.ndarray[list[str]], x_train.shape = (num_messages,))

    Returns:
    all_words_list -- array of all words in both categories;
        (type(all_words_list) -> numpy.ndarray[str], all_words_list.shape = (num_words,))
    ham_words_list -- array of words in 'ham' class;
        (type(ham_words_list) -> numpy.ndarray[str], ham_words_list.shape = (num_words,))
    spam_words_list -- array of words in 'spam' class;
        (type(spam_words_list) -> numpy.ndarray[str], spam_words_list.shape = (num_words,))
    """
    all_words_list = []
    ham_words_list = []
    spam_words_list = []

    ### START CODE HERE ###
    all_words_list = np.concatenate(x_train)
    ham_words_list = np.concatenate(x_train[y_train == 'ham'])
    spam_words_list = np.concatenate(x_train[y_train == 'spam'])
    ### END CODE HERE ###

    return all_words_list, ham_words_list, spam_words_list


all_words_list_a1, ham_words_list_a1, spam_words_list_a1 = categories_words(a1, a2)

In [10]:
print('first five "ham" words of a1: ', ham_words_list_a1[:5])

first five "ham" words of a1:  ['go' 'until' 'jurong' 'point' 'crazy']


**Expected Output**:

<table style="width:40%">
    <tr>
        <td><b>first five "ham" words of a1:</b></td>
       <td> ['go' 'until' 'jurong' 'point' 'crazy'] </td>
    </tr>

</table>

Let's calculate probability of each word in a category. Here you need to use (3) formula.

In [11]:
# Calculate log probability of each word in a category using smoothing

def calc_words_probs_of_category(category_words_list, all_words_list, alpha):
    """
    Returns a dict of log probs words belonging to a category

    Arguments:
    category_words_list -- array of words in category class ("ham" or "spam");
        (type(category_words_arr) -> numpy.ndarray[str], category_words_list.shape = (num_words,))
    all_words_list -- array of all words in both categories;
        (type(all_words_list) -> numpy.ndarray[str], all_words_list.shape = (num_words,))
    alpha -- int number. The smoothing coeficient.

    Returns:
    category_words_dict -- dictionary containing log probability of each word in a category.
        type(category_words_dict) -> dict
    """
    category_words_dict = {}

    ### START CODE HERE ###

    for word in all_words_list:
        category_words_dict[word] = np.log(
            (np.count_nonzero(category_words_list == word) + alpha) / (category_words_list.size + alpha * np.unique(all_words_list).size))

    ### END CODE HERE ###

    return category_words_dict

In [12]:
ham_words_dict_a1 = calc_words_probs_of_category(ham_words_list_a1, all_words_list_a1, alpha=1)
print('The log prob of the word “87121” from the "ham" category in a1:', ham_words_dict_a1['87121'])

spam_words_dict_a1 = calc_words_probs_of_category(spam_words_list_a1, all_words_list_a1, alpha=1)
print('\nThe log prob of the word “87121” from the "spam" category in a1:', spam_words_dict_a1['87121'])

The log prob of the word “87121” from the "ham" category in a1: -4.3694478524670215

The log prob of the word “87121” from the "spam" category in a1: -3.7612001156935624


**Expected Output**:

<table style="width:70%">
    <tr>
        <td><b>The log prob of the word “87121” from the "ham" category in a1:</b></td>
       <td>-4.3694478524670215</td>
    </tr>
    <tr>
        <td><b>The log prob of the word “87121” from the "spam" category in a1:</b></td>
       <td>-3.7612001156935624</td>
    </tr>
    

</table>

Сalculating the prior probability for each class category. Here you need to use (1) formula.

In [13]:
# Calculate prior log probability of each category

def calc_prior_category_prob(y_train):
    """
    Returns prior probabilities of each category

    Arguments:
    y_train -- array which contains list of labels of each message;
        (type(y_train) -> numpy.ndarray[list[str]], x_train.shape = (num_messages,))

    Returns:
    prior_ham_prob -- float number. Prior log probability of "ham" category
    prior_spam_prob -- float number. Prior log probability of "spam" category
    """

    ### START CODE HERE ###
    prior_ham_prob = np.log(np.count_nonzero(y_train == 'ham') / len(y_train))
    prior_spam_prob = np.log(np.count_nonzero(y_train == 'spam') / len(y_train))

    ### END CODE HERE ###

    return prior_ham_prob, prior_spam_prob

In [14]:
print('Prior log probability of "ham" category in a2: ', calc_prior_category_prob(a2)[0])
print('Prior log probability of "spam" category in a2: ', calc_prior_category_prob(a2)[1])

Prior log probability of "ham" category in a2:  -0.40546510810816444
Prior log probability of "spam" category in a2:  -1.0986122886681098


**Expected Output**:

<table style="width:70%">
    <tr>
        <td><b>Prior log probability of "ham" category in a2:</b></td>
       <td>-0.40546510810816</td>
    </tr>
    <tr>
        <td><b>Prior log probability of "spam" category in a2:</b></td>
       <td>-1.0986122886681</td>
    </tr>
    

</table>

### 3.2 Model

In [15]:
class Naive_Bayes(object):
    """
    Parameters:
    -----------
    alpha: int
        The smoothing coeficient.
    """

    def __init__(self, alpha):
        self.alpha = alpha

        self.train_set_x = None
        self.train_set_y = None

        self.all_words_list = []
        self.ham_words_list = []
        self.spam_words_list = []

        self.ham_words_dict = {}
        self.spam_words_dict = {}

        self.prior_ham_prob = None
        self.prior_spam_prob = None

        #You are allowed to create new attributes and methods (but it's not a necessary)

    def fit(self, train_set_x, train_set_y):

        # Generate all_words_list, ham_words_list, spam_words_list using function 'categories_words';
        # Calculate probability of each word in both categories using function 'calc_words_probs_of_category';
        # Calculate prior probability of each category using function 'calc_prior_category_prob'.
        ### START CODE HERE ###

        self.train_set_x = train_set_x
        self.train_set_y = train_set_y
        self.all_words_list, self.ham_words_list, self.spam_words_list = categories_words(train_set_x, train_set_y)
        self.ham_words_dict = calc_words_probs_of_category(self.ham_words_list, self.all_words_list, self.alpha)
        self.spam_words_dict = calc_words_probs_of_category(self.spam_words_list, self.all_words_list, self.alpha)
        self.prior_ham_prob, self.prior_spam_prob = calc_prior_category_prob(train_set_y)

        ### END CODE HERE ###

    def predict(self, test_set_x):

        # Calculate probabilities of belonging to ham and spam category
        # Compare these probabilities and choose the max
        # Return list of predicted labels for test set; type(prediction) -> list, len(prediction) = len(test_set_y)
        ### START CODE HERE ###

        prediction = []
        for message in test_set_x:
            ham_prob = self.prior_ham_prob
            spam_prob = self.prior_spam_prob
            for word in message:
                if word in self.ham_words_dict:
                    ham_prob += self.ham_words_dict[word]
                if word in self.spam_words_dict:
                    spam_prob += self.spam_words_dict[word]
            prediction.append('ham' if ham_prob > spam_prob else 'spam')

        ### END CODE HERE ###
        return prediction

## 4 - Training

First of all, we should define a smoothing coeficient (`alpha`).

In [16]:
a = 1

Now we can initialize our model:

In [17]:
model = Naive_Bayes(alpha=a)

Let's train our model:

In [ ]:
model.fit(train_set_x, train_set_y)

## 5 - Making predictions

In [ ]:
y_predictions = model.predict(test_set_x)

Let's calculate accuracy (accuracy of model must be >0.95):

In [21]:
actual = list(test_set_y)
accuracy = (y_predictions == test_set_y).mean()
print(accuracy)

0.9847533632286996


## 6 - Conclusion
As we can see, our model fits well the hypothesis function to the data.

#### What's next:
1. Try experimenting with the `alpha` to see how this affects the model you have built.
2. Compare the results you have obtained with the `sklearn.naive_bayes.MultinomialNB` model.
3. Try this model in the wild! Select your favorite dataset [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=small&filetype=all&license=all&tagids=13303) and play with it.

##### Naive Bayes Classifier Done!

##### Make sure that you didn't add or delete any notebook cells. Otherwise your work may not be accepted by the validator!